## 0. Install and Import Modules

Ensure `torch` and `torchvision` are installed. If `pytorch3d` is not installed, install it using the following cell:

In [ ]:
# import os
# import sys
# import torch
# need_pytorch3d=False
# try:
#     import pytorch3d
# except ModuleNotFoundError:
#     need_pytorch3d=True
# if need_pytorch3d:
#     if torch.__version__.startswith("1.13.") and sys.platform.startswith("linux"):
#         # We try to install PyTorch3D via a released wheel.
#         pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
#         version_str="".join([
#             f"py3{sys.version_info.minor}_cu",
#             torch.version.cuda.replace(".",""),
#             f"_pyt{pyt_version_str}"
#         ])
#         !pip install fvcore iopath
#         !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
#     else:
#         # We try to install PyTorch3D from source.
#         !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
#         !tar xzf 1.10.0.tar.gz
#         os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
#         !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 KB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 KB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61431 sha256=2a5a735959b1ed1a5c444b3f0a8b8c4fc0d7f2c5ffc6c6c47b9fce8c6fd6784d
  Stored in directory: /root/.cache/pip/wheels/b8/79/07/c0e9367f5b5ea325e246bd73651e8af175fabbef943043b1cc
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31547 sha256=201007029374099283742884f8d86469d1265b3240302b52f894036335fd8d5b
  Stored in directory: /root/.cache/pip/wheels/89/3e/24/0f349c0b2eeb6965903035f3b00dbb5c9bea437b4a2f18d82c
Successfully built fvcore iopath
Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py38_cu116_pyt1131/dow

In [ ]:
# # imports
# import numpy as np
# import torch
# from pytorch3d.transforms.so3 import (
#     so3_exp_map,
#     so3_relative_angle,
# )
# from pytorch3d.transforms.se3 import (
#     se3_exp_map,
#     se3_log_map,
# )

# import pytorch3d.transforms as transforms
    
# # add path for demo utils
# import sys
# import os
# sys.path.append(os.path.abspath(''))

# # set for reproducibility
# torch.manual_seed(42)
# if torch.cuda.is_available():
#     device = torch.device("cuda:0")
# else:
#     device = torch.device("cpu")
#     print("WARNING: CPU only, this will be slow!")

## **Data preparation**

In [ ]:
# # Load the matrices 
# T_o_cb_batch = np.load('')
# batch_size = T_o_cb_batch.shape[0]
# T_o_p_batch = np.load('')
# T_c_p_batch = np.load('')
# X_matrix = np.load('')
# X_batch = np.tile(X_matrix, [batch_size,1,1])

# # Load into torch tensor
# X_batch = torch.tensor(X_batch)
# T_o_p_batch = torch.tensor(T_o_p_batch)
# T_o_cb_batch = torch.tensor(T_o_cb_batch)
# T_c_p_batch = torch.tensor(T_c_p_batch)

In [ ]:
# ## test
# from scipy.spatial.transform import Rotation as R

# batch_size = 10
# T_o_cb_batch = np.tile(np.eye(4), [batch_size,1,1])
# T_o_p_batch = np.tile(np.eye(4), [batch_size,1,1])
# T_c_p_batch = np.tile(np.eye(4), [batch_size,1,1])
# X_matrix = np.hstack((np.vstack((R.random().as_matrix(), np.zeros([1,3]))), np.ones([4,1])))
# X_batch = np.tile(X_matrix, [batch_size,1,1])
# X_batch[:, :3, 3] = 2
# print(X_batch[1, :,:])
# X_batch = torch.tensor(X_batch)
# T_o_p_batch = torch.tensor(T_o_p_batch)
# T_o_cb_batch = torch.tensor(T_o_cb_batch)
# T_c_p_batch = torch.tensor(T_c_p_batch)

[[-0.49508795 -0.8619929  -0.10888599  2.        ]
 [-0.86414445  0.47551297  0.16474763  2.        ]
 [-0.09023459  0.17565779 -0.98030712  2.        ]
 [ 0.          0.          0.          1.        ]]


In [ ]:
# # Create Transform3d objects from the batch of transformation matrices
# T_o_cb = transforms.Transform3d(matrix=T_o_cb_batch, device=device)
# X = transforms.Transform3d(matrix=X_batch, device=device)
# T_o_p = transforms.Transform3d(matrix=T_o_p_batch, device=device)
# T_c_p = transforms.Transform3d(matrix=T_c_p_batch, device=device)

# # cam_gt: the ground truth camera pose w.r.t the optical tracker cam_gt = T_o_p * (T_c_p)^-1
# cam_gt = T_o_cb.compose(X)

# # cam_real: the camera pose from initial hand-eye calibration cam_real = T_o_cb * X
# cam_real = T_o_p.compose(T_c_p.inverse())

## **Define optimization loss function**



In [ ]:
# def camera_distance(hand_eye_real, hand_eye_gt):
#   '''
#   Calculate the l2 distance in se(3) manifold for the camera.
#   distance = sum||log(hand_eye_real*hand_eye_gt)||_2
#   '''

#   # Convert a batch of 4x4 transformation matrices transform to 
#   # a batch of 6-dimensional SE(3) logarithms of the SE(3) matrices
#   # print(hand_eye_real.compose(hand_eye_gt.inverse()).get_matrix())
#   tmp_matrix = hand_eye_real.compose(hand_eye_gt.inverse()).get_matrix()
#   vec = se3_log_map(tmp_matrix.permute(0,2,1)).sum(0)
#   # print(torch.norm(vec))

#   return torch.norm(vec)
  

# # camera_distance(cam_real, cam_gt)

## **Optimization**

In [ ]:
# # Create Transform3d objects from the batch of transformation matrices
# T_o_cb = transforms.Transform3d(matrix=T_o_cb_batch, device=device)
# X = transforms.Transform3d(matrix=X_batch, device=device)
# T_o_p = transforms.Transform3d(matrix=T_o_p_batch, device=device)
# T_c_p = transforms.Transform3d(matrix=T_c_p_batch, device=device)

# # hand_eye_real: the hand-eye transformation from initial calibration X
# hand_eye_real_6D = se3_log_map(X.get_matrix().permute(0, 2, 1))
# hand_eye_real_6D.requires_grad = True


# # hand_eye_gt: the hand-eye transformation from the chain 
# # hand_eye_gt = (T_o_cb)^-1 * T_o_p * (T_c_p)^-1
# hand_eye_gt = T_o_cb.inverse().compose(T_o_p).compose(T_c_p.inverse())

# # init the optimizer
# optimizer = torch.optim.SGD([hand_eye_real_6D], lr=.001, momentum=0.9)

# # run the optimization
# n_iter = 2000  # fix the number of iterations
# for it in range(n_iter):
#     # re-init the optimizer gradients
#     optimizer.zero_grad()

#     # transform the 6d vector into 4x4 matrix
#     hand_eye_real = se3_exp_map(hand_eye_real_6D).permute(0, 2, 1)
#     hand_eye_real = transforms.Transform3d(matrix=hand_eye_real, device=device)

#     # compare the composed cameras with the ground truth relative cameras
#     # camera_distance corresponds to $d$ from the description
#     loss = \
#         camera_distance(hand_eye_real, hand_eye_gt)
#     # loss.requires_grad_(True)

#     # our loss function is the camera_distance
#     loss.backward()
    
#     # apply the gradients
#     optimizer.step()

#     # # plot and print status message
#     if it % 200==0 or it==n_iter-1:
#         status = 'iteration=%3d; camera_distance=%1.3e' % (it, loss)
#         print(status)
        
# print('Optimization finished.')

iteration=  0; camera_distance=4.696e+01
iteration=200; camera_distance=2.786e+01
iteration=400; camera_distance=7.859e+00
iteration=600; camera_distance=5.260e-04
iteration=800; camera_distance=1.630e-02
iteration=1000; camera_distance=7.543e-03
iteration=1200; camera_distance=1.241e-02
iteration=1400; camera_distance=7.818e-03
iteration=1600; camera_distance=1.311e-02
iteration=1800; camera_distance=3.938e-03
iteration=1999; camera_distance=4.000e-03
Optimization finished.


## **Pypose implementation**

In [ ]:
# !pip install pypose

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 KB 10.9 MB/s eta 0:00:00


In [21]:
import os
import sys
import numpy as np
import torch, pypose as pp
from torch import nn
from scipy.spatial.transform import Rotation as R

In [22]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    print("WARNING: CPU only, this will be slow!")

## **Data Preparation**

In [ ]:
T_c_p_batch = np.load(os.path.join(params_save_dir, 'T_c_p_opti3.npy'))
T_c_p_batch_zed = np.load(os.path.join(params_save_dir, 'T_c_p_opti3zed.npy'))

T_c_p = pp.mat2SE3(T_c_p_batch)
T_c_p_zed = pp.mat2SE3(T_c_p_batch_zed)
# T_c_p @ pp.Inv(T_c_p_zed)
print([T_c_p[i][:3].norm().item() for i in range(160)],'\n',[T_c_p_zed[i][:3].norm().item() for i in range(41)])
# print(T_c_p_batch[0],'\n',T_c_p_batch_zed[0])

In [92]:
# # Load the matrices
params_save_dir = '../data/post_optimization_data/'
T_o_cb_batch = np.load(os.path.join(params_save_dir, 'T_o_cb_opti3.npy'))[:150]
T_o_cb_batch[:,:3, 3] /= 1000 
batch_size = T_o_cb_batch.shape[0]
T_o_p_batch = np.load(os.path.join(params_save_dir, 'T_o_p_opti3.npy'))[:150]
T_o_p_batch[:,:3, 3] /= 1000
T_c_p_batch = np.load(os.path.join(params_save_dir, 'T_c_p_opti3zed.npy'))[:150]
T_c_p_batch[:,:3, 3] /= 1000
X_matrix = np.load('../params/hand_eye_X_0220.npy')
X_matrix[:3, 3] /= 1000
# X_batch = np.tile(X_matrix, [batch_size,1,1])[:40]

# convert to pp tensor
X = pp.mat2SE3(X_matrix)
T_o_p = pp.mat2SE3(T_o_p_batch)
T_o_cb = pp.mat2SE3(T_o_cb_batch)
T_c_p = pp.mat2SE3(T_c_p_batch)

# print(np.linalg.norm(X_batch[1, :3,3]))
print(X[:3].norm())

tensor(0.1243, dtype=torch.float64)


In [316]:
# # test
# batch_size = 800
# T_o_cb_batch = np.tile(np.eye(4), [batch_size,1,1])
# T_o_p_batch = np.tile(np.eye(4), [batch_size,1,1])
# T_c_p_batch = np.tile(np.eye(4), [batch_size,1,1])
# X_matrix = np.hstack((np.vstack((R.random().as_matrix(), np.zeros([1,3]))), np.ones([4,1])))
# X_batch = np.tile(X_matrix, [batch_size,1,1])
# X_batch[:, :3, 3] = 2.5

# print(X_batch[1, :,:])

# # convert to pp tensor
# X = pp.mat2SE3(X_batch)
# T_o_p = pp.mat2SE3(T_o_p_batch)
# T_o_cb = pp.mat2SE3(T_o_cb_batch)
# T_c_p = pp.mat2SE3(T_c_p_batch)
# # print(X_batch[1,...])
# # print(X_batch @ pp.Inv(X_batch))

## **Optimization**

In [123]:
class CamDis(nn.Module):
    def __init__(self, hand_eye_real_6D):
        super().__init__()
        self.real = pp.Parameter(hand_eye_real_6D)

    def forward(self, input):
        # input hand_eye_gt
        hand_eye_real = self.real.Exp()
        tmp_quats = pp.Inv(hand_eye_real) @ input
        vec = tmp_quats.Log()
        t = tmp_quats[:, :3].mean(0)
        rot_vec = vec[:, 3:].mean(0)
        # print(self.real.Exp())
        return t.norm() + rot_vec.norm()

In [112]:
hand_eye_gt = pp.Inv(T_o_cb) @ T_o_p @ pp.Inv(T_c_p)
hand_eye_gt = hand_eye_gt.to('cpu')
hand_eye_real = X
# hand_eye_gt[:3] = hand_eye_gt[:3]/1000
print(hand_eye_gt.shape)
print(X[:3].norm())
print([hand_eye_gt[i,:3].norm().item() for i in range(40)])
print(pp.Inv(hand_eye_real)*hand_eye_gt)
# print(hand_eye_real*pp.Inv(hand_eye_gt))
test = pp.Inv(hand_eye_real)*hand_eye_gt
print(test[:, :3].mean(0))


torch.Size([150, 7])
tensor(0.1243, dtype=torch.float64)
[0.12938696244371226, 0.13031321266780965, 0.13122056046503783, 0.13074747792270724, 0.13093554471659627, 0.1312828602486734, 0.13171319895265735, 0.13263997988197918, 0.1340988850882946, 0.132123919294936, 0.13232468656155988, 0.13100444750158227, 0.13147318927977159, 0.1317402825657335, 0.13103063831723974, 0.1310387568863373, 0.13298470118556988, 0.1350633595663893, 0.1422564702940825, 0.14432143905572603, 0.12745192506317815, 0.1283203116317409, 0.12929842102438305, 0.13152221597077932, 0.12657259807956583, 0.1297832769391131, 0.1337537602883679, 0.13298695228159113, 0.13369253823686397, 0.12979472946272366, 0.13103521025844286, 0.13019661757430978, 0.13079871061901846, 0.13030468638285467, 0.13056573388299272, 0.12927139947572194, 0.12959650309009238, 0.12809534649169174, 0.12690133237520276, 0.12812610845788866]
SE3Type LieTensor:
LieTensor([[ 6.2738e-03,  3.4432e-04,  7.4286e-03,  ..., -8.7473e-03,
             3.2501e-03,

In [128]:
# hand_eye_real: the hand-eye transformation from initial calibration X

hand_eye_real = X.clone().detach()
hand_eye_real_6D = hand_eye_real.Log()
print(hand_eye_real)
# hand_eye_gt: the hand-eye transformation from the chain 
# hand_eye_gt = (T_o_cb)^-1 * T_o_p * (T_c_p)^-1
hand_eye_gt = pp.Inv(T_o_cb) @ T_o_p @ pp.Inv(T_c_p)
hand_eye_gt = hand_eye_gt.clone().detach()
# print(np.linalg.norm(hand_eye_gt[4].matrix()[:3,3]))
# print(np.linalg.norm(X[0].matrix()[:3,3]))

# init the optimizer
camera_distance = CamDis(hand_eye_real_6D).to(device)
input = hand_eye_gt.to(device)
strategy = pp.optim.strategy.Adaptive(damping=2e-6)
optimizer = pp.optim.LM(camera_distance, strategy=strategy)
# run the optimization
n_iter = 2000  # fix the number of iterations
for it in range(n_iter):
    loss = optimizer.step(input)
    # # plot and print status message
    if it % 100==0 or it==n_iter-1:
        status = 'iteration=%3d; camera_distance=%1.3e' % (it, loss)
        print(status)
        # print(camera_distance.real.Exp())
    if loss < 1e-4:
        print('Early Stopping with loss:', loss.item())
        break
res = camera_distance.real.Exp()
print('The final result:\n', res)
print('Optimization finished.')


SE3Type LieTensor:
LieTensor([ 0.0202,  0.0986,  0.0730, -0.2865,  0.5003,  0.3674,  0.7298],
          device='cuda:0', dtype=torch.float64)
iteration=  0; camera_distance=6.520e+00
iteration=100; camera_distance=6.504e+00
iteration=200; camera_distance=6.501e+00
iteration=300; camera_distance=6.498e+00
iteration=400; camera_distance=6.495e+00
iteration=500; camera_distance=6.492e+00
iteration=600; camera_distance=6.490e+00
iteration=700; camera_distance=6.487e+00
iteration=800; camera_distance=6.484e+00
iteration=900; camera_distance=6.481e+00
iteration=1000; camera_distance=6.478e+00
iteration=1100; camera_distance=6.475e+00
iteration=1200; camera_distance=6.473e+00
iteration=1300; camera_distance=6.473e+00
iteration=1400; camera_distance=6.473e+00
iteration=1500; camera_distance=6.473e+00
iteration=1600; camera_distance=6.473e+00
iteration=1700; camera_distance=6.473e+00
iteration=1800; camera_distance=6.473e+00
iteration=1900; camera_distance=6.473e+00
iteration=1999; camera_dista

In [129]:
X = X.to(device)
# print(input[0,:3].norm())
print(X)
print(res)
print(X[:3].norm())
print(res[:3].norm())


SE3Type LieTensor:
LieTensor([ 0.0202,  0.0986,  0.0730, -0.2865,  0.5003,  0.3674,  0.7298],
          device='cuda:0', dtype=torch.float64)
SE3Type LieTensor:
LieTensor([ 0.1747,  0.1768,  0.0062, -0.2676,  0.5513,  0.3418,  0.7125],
          device='cuda:0', dtype=torch.float64, grad_fn=<AliasBackward0>)
tensor(0.1243, device='cuda:0', dtype=torch.float64)
tensor(0.2486, device='cuda:0', dtype=torch.float64, grad_fn=<NormBackward1>)


In [60]:

print(pp.matrix(X),'\n', pp.matrix(pp.Inv(res))@ pp.matrix(X))

SE3Type LieTensor:
LieTensor([ 13.8470, 109.8871,  76.9371,  -0.7170,  -0.3874,   0.5053,  -0.2838],
          device='cuda:0', dtype=torch.float64) SE3Type LieTensor:
LieTensor([20.2430, 98.5587, 72.9542, -0.2865,  0.5003,  0.3674,  0.7298],
          device='cuda:0', dtype=torch.float64)
tensor([[ 2.2935e-01, -8.2292e-01,  5.1981e-01,  2.0243e+01],
        [ 2.4962e-01,  5.6591e-01,  7.8577e-01,  9.8559e+01],
        [-9.4079e-01, -5.0468e-02,  3.3521e-01,  7.2954e+01],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]], device='cuda:0',
       dtype=torch.float64) 
 tensor([[ 9.9900e-01,  4.4030e-02, -7.1768e-03,  1.9283e+00],
        [ 4.4164e-02, -9.9883e-01,  1.9741e-02,  1.1429e+01],
        [-6.2993e-03, -2.0038e-02, -9.9978e-01,  7.1253e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]], device='cuda:0',
       dtype=torch.float64)


In [62]:
handeyeX = np.load('hand_eye_X.npy')
handeyeX

array([[ 2.29353474e-01, -8.22919865e-01,  5.19807541e-01,
         2.02430249e+01],
       [ 2.49615163e-01,  5.65911589e-01,  7.85771178e-01,
         9.85586502e+01],
       [-9.40791823e-01, -5.04675049e-02,  3.35207065e-01,
         7.29542443e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])